In [17]:
import pandas as pd
import numpy as np

import joblib
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
rf = joblib.load('/home/ejenliya/Projects/FotoCaptions/models/random_forest_clf.pkl')
features = rf.feature_names_in_.tolist()

In [14]:
#example of test data
test_data = pd.read_csv('/home/ejenliya/Projects/FotoCaptions/data/data.csv')
test_data = test_data[test_data['likes'] != 'другим']
test_data = test_data.dropna().iloc[:10]
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 12
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    10 non-null     object
 1   likes   10 non-null     object
 2   time    10 non-null     object
dtypes: object(3)
memory usage: 320.0+ bytes


In [18]:
def filter_text(str):
    str = str.lower()
    new_str = ''
    ascii_numbers = list(range(1072, 1106))
    ascii_numbers.append(32)

    for symbol in str:
        if ord(symbol) in ascii_numbers:
            new_str += symbol 
        else:
            new_str += ' '

    return new_str

import pymorphy2


morth = pymorphy2.MorphAnalyzer()
functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'}

def drop_superfuluos(text):
    text = text.split()
    new_text = ''

    for word in text:
        if morth.parse(word)[0].tag.POS not in functors_pos:
            new_text += morth.parse(word)[0].normal_form
            new_text += ' '

    return new_text

In [19]:
test_data = test_data['text'].apply(filter_text)
test_data = test_data.apply(drop_superfuluos)
test_data.head()

0    органайзер шея специальный держатель карта док...
1    хибины актив насеверёжить активныйотдых хибины...
2    дата тур март х дневный тур март март март цен...
3    там быть таинственно тихо звук никто зима снег...
4    это ехать точно стоить вы собраться монастырь ...
Name: text, dtype: object

In [20]:
tf_idf_vec = TfidfVectorizer(use_idf=True,  
                        smooth_idf=True,  
                        ngram_range=(1,1))
tf_idf_data = tf_idf_vec.fit_transform(test_data) 
tf_idf_frame = pd.DataFrame(tf_idf_data.toarray(), columns=tf_idf_vec.get_feature_names_out())

tf_idf_frame.head()

,абакан,адрес,актив,активножитьнезапретить,активныйотдых,алмазный,алматы,арктика,атмосфера,бег,...,черёмушки,чимбулак,чистый,шапка,шея,экскурсия,электро,эндрютво,это,южнаякорея
0,0.0,0.0,0.000000,0.000000,0.000000,0.139054,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.139054,0.000000,0.0,0.000000,0.000000,0.0
1,0.0,0.0,0.193701,0.193701,0.193701,0.000000,0.0,0.193701,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.193701,0.000000,0.0
2,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.221143,0.0,0.000000,0.000000,0.0
3,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
4,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.164402,0.0


In [32]:
valid_features = set(tf_idf_frame.columns).intersection(set(features))
valid_features

{'быть',
 'время',
 'гора',
 'дорога',
 'друг',
 'зима',
 'можно',
 'море',
 'мы',
 'парк',
 'природа',
 'путешествие',
 'россия',
 'смотреть',
 'снег',
 'спорт',
 'туризм',
 'фото'}

In [36]:
frame_for_prediction = pd.DataFrame(np.random.randint(0, 1, size=(test_data.shape[0], len(features))), columns=features).astype(float)
frame_for_prediction.head()

,чилла,набережная,фото,сюрприз,свидание,цветок,парк,зима,мы,выпускной,...,этот,вечерний,ночь,следуйзамный,россия,ул,море,ребёнок,вкус,москва
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
for i in range(tf_idf_frame.shape[0]):
    for col in valid_features:
        try:
            frame_for_prediction.loc[i][col] = tf_idf_frame.loc[i][col]
        except Exception as ex:
            print('failed at adding')

In [43]:
preds = rf.predict_proba(frame_for_prediction)
preds

array([[0.51528319, 0.48471681],
       [0.22166667, 0.77833333],
       [0.59622461, 0.40377539],
       [0.36666667, 0.63333333],
       [0.67988889, 0.32011111],
       [0.22666667, 0.77333333],
       [0.16666667, 0.83333333],
       [0.26217997, 0.73782003],
       [0.28      , 0.72      ],
       [0.30666667, 0.69333333]])

In [47]:
xgb = joblib.load('/home/ejenliya/Projects/FotoCaptions/models/xgboost_clf.pkl')

In [46]:
preds_xgb = xgb.predict_proba(frame_for_prediction)
preds_xgb

array([[0.43625492, 0.5637451 ],
       [0.27130026, 0.72869974],
       [0.55113274, 0.44886726],
       [0.35035205, 0.64964795],
       [0.89116585, 0.10883416],
       [0.2426092 , 0.7573908 ],
       [0.24040532, 0.7595947 ],
       [0.19840533, 0.8015947 ],
       [0.25605088, 0.7439491 ],
       [0.30478817, 0.6952118 ]], dtype=float32)